In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.multiprocessing as mp
from FPN import Features, FPNetwork , classificationhead , bboxhead
from Loss import Lossfunction
from datasets import load_dataset
import torch.optim as optim
from torch.cuda.amp import  GradScaler
from torch.amp import autocast
import gc
from dataset_convert import AnchorGenerator, FaceDetectionDataset
import random
scaler = GradScaler()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mp.set_start_method('spawn', force=True)

print(device)

/home/faces2.0/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


/tmp/ipykernel_772/3959396688.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [2]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)

resnet_backbone = nn.Sequential(*list(resnet50.children())[:-2]).to(device)
layers_r50= {'4.0':256,  #-> (160,160)
             '4.1':512,  #-> (80,80)
             '4.2':1024, #-> (40,40)
             '4.3':2048  #-> (20,20)
             }

/home/faces2.0/.conda/lib/python3.12/site-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(


Downloading: "https://github.com/NVIDIA/DeepLearningExamples/zipball/torchhub" to /home/.cache/torch/hub/torchhub.zip


/home/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


Downloading: "https://api.ngc.nvidia.com/v2/models/nvidia/resnet50_pyt_amp/versions/20.06.0/files/nvidia_resnet50_200821.pth.tar" to /home/.cache/torch/hub/checkpoints/nvidia_resnet50_200821.pth.tar


100%|██████████| 97.7M/97.7M [00:01<00:00, 91.4MB/s]


In [3]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model = model.features.to(device)

Using cache found in /Users/vipulagarwal/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
dataset = load_dataset("CUHK-CSE/wider_face")
train_dataset = dataset['train'].with_format("torch")
val_dataset = dataset['validation'].with_format("torch")

Generating validation split: 100%|██████████| 3226/3226 [00:00<00:00, 5392.09 examples/s]


In [4]:
extractor = Features(resnet_backbone,layers=list(layers_r50.keys()))
topdown = FPNetwork(in_channels=layers_r50, out_channels=256)
classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
bboxregression = bboxhead(channels= 256 , num_anchors= 12)
loss =Lossfunction(lambd=1)
anchors = AnchorGenerator()
data = FaceDetectionDataset(train_dataset,anchors)
vali_data = FaceDetectionDataset(val_dataset,anchors)

In [6]:
# extractor = Features(model,['3','6', '13','18'])
# topdown = FPNetwork(out_channels=256)
# classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
# bboxregression = bboxhead(channels= 256 , num_anchors= 12)
# loss =Lossfunction(lambd=10)
# anchors = AnchorGenerator()
# data = FaceDetectionDataset(train_dataset,anchors)
# vali_data = FaceDetectionDataset(val_dataset,anchors)

In [7]:
torch.cuda.empty_cache()
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

Allocated: 0.00 GB
Cached: 0.00 GB


In [5]:
def forward(p):
    features = extractor.extract(p)
    newfeatures = topdown(features)
    output = {}
    for key in list(newfeatures.keys()):
        temp = {}
        temp["bbox"] = bboxregression(newfeatures[key])
        temp["cls"] = classifier(newfeatures[key])
        output[key] = temp
    return output

In [18]:
i=random.randint(0,1200)
kk =data[i][0].unsqueeze(0).to(device)
jk = forward(kk)
ll = loss(jk, data[i][1])
ll

tensor(3.7871, device='cuda:0', grad_fn=<AddBackward0>)

In [31]:
(jk['4.3']['cls'])[:,2400:2800]

tensor([[[-3.6243],
         [-3.5365],
         [-3.9273],
         [-3.4183],
         [-3.9731],
         [-3.7400],
         [-3.8969],
         [-3.8171],
         [-3.6029],
         [-3.2696],
         [-3.3121],
         [-3.5310],
         [-3.3870],
         [-3.3835],
         [-3.3399],
         [-3.4736],
         [-3.6334],
         [-3.8824],
         [-3.5717],
         [-3.1045],
         [-3.5244],
         [-3.5147],
         [-3.6700],
         [-3.6975],
         [-3.8044],
         [-4.3616],
         [-3.8088],
         [-3.9572],
         [-3.3879],
         [-3.8553],
         [-4.2100],
         [-3.6968],
         [-3.3975],
         [-3.6694],
         [-3.8204],
         [-3.8736],
         [-3.6786],
         [-3.9293],
         [-3.8306],
         [-3.4526],
         [-3.3792],
         [-3.3392],
         [-3.3587],
         [-3.3264],
         [-3.8113],
         [-4.0264],
         [-3.8518],
         [-3.7846],
         [-2.9091],
         [-3.5213],


In [19]:
i

1038

In [12]:
# for i in range(0,100):
#     data[i][0]

In [8]:
total_params=sum(p.numel() for p in extractor.parameters())
total_params+= sum(p.numel() for p in topdown.parameters())
total_params+= sum(p.numel() for p in classifier.parameters())
total_params+= sum(p.numel() for p in bboxregression.parameters())

print(total_params)

31719548


In [9]:
learning_rate = 1e-4

def train_with_accumulation(epochs: int, training_data, validation_data=None, accumulation_steps=4):
    # Set models to training mode
    extractor.train()
    topdown.train()
    classifier.train()
    bboxregression.train()
    
    if hasattr(extractor, 'gradient_checkpointing_enable'):
        extractor.gradient_checkpointing_enable()
    if hasattr(topdown, 'gradient_checkpointing_enable'):
        topdown.gradient_checkpointing_enable()
    
    optimizer = optim.Adam(
        list(extractor.parameters()) +
        list(topdown.parameters()) +
        list(classifier.parameters()) +
        list(bboxregression.parameters()), 
        lr=learning_rate
    )
    
    loss_history = {}
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        batch_count = 0
        running_loss = 0.0
        
        print(f"\nEpoch {epoch + 1}/{epochs}")
        print("-" * 30)
        try:
            for batch_idx, (image, bbox) in enumerate(training_data):
                optimizer.zero_grad()
                
                with autocast('cuda'):
                    model_pred = forward(image.cuda())
                    ll = loss(model_pred, bbox) / accumulation_steps  # Scale loss
                scaler.scale(ll).backward()
                
                if (batch_idx + 1) % accumulation_steps == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                
                batch_loss = ll.item() * accumulation_steps  # Unscale for logging
                epoch_loss += batch_loss
                running_loss += batch_loss
                batch_count += 1
                
                if (batch_idx + 1) % 10 == 0:
                    avg_running_loss = running_loss / 10
                    print(f"Batch {batch_idx + 1}: Avg Loss = {avg_running_loss:.6f}")
                    running_loss = 0.0

                # Memory 
                del image , model_pred , bbox
                del ll
                
                torch.cuda.empty_cache()
                gc.collect()
        except Exception as e :
            print (e)
            print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
            print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
            torch.cuda.empty_cache()
            print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
            print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
            break

        except KeyboardInterrupt:
            print("INTERRUPTED!!")
            return loss_history   

                
        if batch_count % accumulation_steps != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        avg_epoch_loss = epoch_loss / batch_count if batch_count > 0 else 0
        loss_history[epoch] = avg_epoch_loss
        print("-" * 30)
        print(f"Epoch {epoch + 1} Average Loss: {avg_epoch_loss:.6f}")
        print("-" * 30)
        
        
        # Cleanup at epoch end
        gc.collect()
        torch.cuda.empty_cache()
    
    return loss_history



torch.cuda.empty_cache()
batch_size = 8
epochs = 5
training_data = DataLoader(
    data,
    batch_size=batch_size, 
    num_workers=4,     
    pin_memory=True,        
    persistent_workers=True, 
    prefetch_factor=4,       
    drop_last=True,
    shuffle=True
)
validation_data = DataLoader(vali_data,
    batch_size=batch_size, 
    num_workers=4,     
    pin_memory=True,         
    persistent_workers=True, 
    prefetch_factor=4,       
    drop_last=True,
    shuffle=True)

In [10]:
lossdata = train_with_accumulation(epochs = 1, training_data = training_data)


Epoch 1/1
------------------------------
Batch 10: Avg Loss = 101.507450
Batch 20: Avg Loss = 191.673210
Batch 30: Avg Loss = 106.350500
Batch 40: Avg Loss = 57.371157
Batch 50: Avg Loss = 43.992403
Batch 60: Avg Loss = 37.837786
Batch 70: Avg Loss = 26.042994
Batch 80: Avg Loss = 15.267632
Batch 90: Avg Loss = 9.869727
Batch 100: Avg Loss = 11.371710
Batch 110: Avg Loss = 7.267350
Batch 120: Avg Loss = 55.859939
Batch 130: Avg Loss = 6.483840
Batch 140: Avg Loss = 6.131270
Batch 150: Avg Loss = 5.733312
Batch 160: Avg Loss = 5.793077
Batch 170: Avg Loss = 5.104185
Batch 180: Avg Loss = 5.244019
Batch 190: Avg Loss = 5.447075
Batch 200: Avg Loss = 5.235138
Batch 210: Avg Loss = 4.853832
Batch 220: Avg Loss = 4.898820
Batch 230: Avg Loss = 17.053326
Batch 240: Avg Loss = 4.743194
Batch 250: Avg Loss = 5.727100
Batch 260: Avg Loss = 4.524421
Batch 270: Avg Loss = 4.521035
Batch 280: Avg Loss = 4.662802
Batch 290: Avg Loss = 4.571929
Batch 300: Avg Loss = 13.792609
Batch 310: Avg Loss = 

In [ ]:
loss = Lossfunction()
extractor.eval()
topdown.eval()
classifier.eval()
bboxregression.eval()
accumulation_steps = 4
for batch_idx, (image, bbox) in enumerate(validation_data):

    with autocast('cuda'):
        model_pred = forward(image.cuda())
        ll = loss(model_pred, bbox) / accumulation_steps  # Scale loss
    del image , model_pred , bbox
    print(ll)

In [19]:
torch.cuda.empty_cache()
print(f"Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

Allocated: 2.11 GB
Cached: 2.21 GB


In [ ]:
#   git config --global user.email "agarwalvipul577@gmail.com"
#   git config --global user.name "VIPUL577"

In [20]:
data[1000][0].unsqueeze(0).shape

torch.Size([1, 3, 640, 640])

In [13]:
def check(i, threshold , layers = ['4.0','4.1','4.2','4.3']):
    s=forward(vali_data[i][0].unsqueeze(0).cuda())
    t= data[i][1]
    ls = loss(s,t)
    print(f"Loss:{ls}")
    for key,layer in enumerate(layers):
        print(f"prediction: {(nn.Sigmoid()(s[layer]['cls'].cpu())>threshold).sum()} out of total {s[layer]['cls'].cpu().shape} ")
        print(f"truth: {t[len(layers)-key-1]['cls_targets'].sum()} out of total {t[len(layers)-key-1]['cls_targets'].shape}")

In [16]:
check(1080, 0.5)

Loss:15410.435546875
prediction: 149654 out of total torch.Size([1, 307200, 1]) 
truth: 32 out of total torch.Size([1, 307200, 1])
prediction: 37679 out of total torch.Size([1, 76800, 1]) 
truth: 3 out of total torch.Size([1, 76800, 1])
prediction: 9439 out of total torch.Size([1, 19200, 1]) 
truth: 0 out of total torch.Size([1, 19200, 1])
prediction: 2369 out of total torch.Size([1, 4800, 1]) 
truth: 0 out of total torch.Size([1, 4800, 1])


In [17]:
check(1081, 0.5)


Loss:93.85011291503906
prediction: 5641 out of total torch.Size([1, 307200, 1]) 
truth: 352 out of total torch.Size([1, 307200, 1])
prediction: 1418 out of total torch.Size([1, 76800, 1]) 
truth: 127 out of total torch.Size([1, 76800, 1])
prediction: 367 out of total torch.Size([1, 19200, 1]) 
truth: 13 out of total torch.Size([1, 19200, 1])
prediction: 91 out of total torch.Size([1, 4800, 1]) 
truth: 0 out of total torch.Size([1, 4800, 1])


In [17]:
check(1081, 0.9)


Loss:3.1815199851989746
prediction: 0 out of total torch.Size([1, 307200, 1]) 
truth: 352 out of total torch.Size([1, 307200, 1])
prediction: 0 out of total torch.Size([1, 76800, 1]) 
truth: 127 out of total torch.Size([1, 76800, 1])
prediction: 0 out of total torch.Size([1, 19200, 1]) 
truth: 13 out of total torch.Size([1, 19200, 1])
prediction: 0 out of total torch.Size([1, 4800, 1]) 
truth: 0 out of total torch.Size([1, 4800, 1])


In [21]:
torch.save(extractor.state_dict(),'/home/faces2.0/models/extractor.pt')
torch.save(topdown.state_dict(),'/home/faces2.0/models/topdown.pt')
torch.save(classifier.state_dict(),'/home/faces2.0/models/classifier.pt')
torch.save(bboxregression.state_dict(),'/home/faces2.0/models/bboxregression.pt')

